---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
#import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    #nx.draw(P1_Graphs[4], node_size=1)
    #plt.show()
    # 0 PA
    # 1 Sw_H
    # 2 SW_L
    # 3 SW_L
    # 4 PA
    
    return ['PA','SW_L','SW_L','PA','SW_H']
#graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [44]:
from sklearn.svm import SVC

def salary_predictions():
    
    df = pd.DataFrame(G.nodes(data=True))
    
    df['ManagementSalary'] = df[1].apply(lambda x:x['ManagementSalary'])
    
    # add clustering
    df['clustering'] = df[0].apply(lambda x:nx.clustering(G,x))
    
    # add degree
    df['degree'] = df[0].apply(lambda x:nx.degree(G,x))

    # cleanup
    df.drop([0,1],axis=1,inplace=True)
    
    # split into test/train
    train = df.dropna()
    X_train = train[['clustering','degree']]
    y_train = train[['ManagementSalary']]
    
    X_test =  df[df['ManagementSalary'].isnull()]
    X_test.drop(['ManagementSalary'],axis=1,inplace=True)
              
    clf = SVC(kernel="linear", probability=True)
    
    
    clf.fit(X_train,y_train)
    
    X_test['proba'] = clf.predict_proba(X_test)[:,1]
    
    return X_test['proba']
#salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1       0.126296
2       0.604840
5       0.974826
8       0.115724
14      0.342498
18      0.169089
27      0.273816
30      0.358927
31      0.184154
34      0.163647
37      0.109445
40      0.238262
45      0.148485
54      0.266431
55      0.203561
60      0.219796
62      0.999987
65      0.437015
77      0.062484
79      0.106212
97      0.095817
101     0.052290
103     0.194960
108     0.098624
113     0.500000
122     0.050287
141     0.414906
142     0.931368
144     0.060728
145     0.390877
          ...   
913     0.048141
914     0.084343
915     0.003787
918     0.147238
923     0.058536
926     0.094561
931     0.058622
934     0.015383
939     0.014606
944     0.003787
945     0.055652
947     0.202937
950     0.071695
951     0.055653
953     0.027744
959     0.003787
962     0.003989
963     0.120958
968     0.157129
969     0.143853
974     0.096534
984     0.014606
987     0.119111
989     0.157129
991     0.157129
992     0.004202
994     0.003787
996     0.0037

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [46]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

            Future Connection
(6, 840)                  0.0
(4, 197)                  0.0
(620, 979)                0.0
(519, 872)                0.0
(382, 423)                0.0
(97, 226)                 1.0
(349, 905)                0.0
(429, 860)                0.0
(309, 989)                0.0
(468, 880)                0.0
(228, 715)                0.0
(397, 488)                0.0
(253, 570)                0.0
(435, 791)                0.0
(711, 737)                0.0
(263, 884)                0.0
(342, 473)                1.0
(523, 941)                0.0
(157, 189)                0.0
(542, 757)                0.0
(731, 870)                0.0
(497, 973)                0.0
(93, 398)                 0.0
(102, 604)                0.0
(206, 303)                1.0
(57, 447)                 0.0
(417, 758)                0.0
(834, 837)                0.0
(261, 557)                0.0
(514, 740)                0.0
...                       ...
(672, 848)                NaN
(28, 127) 

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [114]:
from sklearn.linear_model import LogisticRegression

def new_connections_predictions():

    enhanced = future_connections.copy()
    
    # Add features
    # M1: Common neighbors
    enhanced["Common_Neighbors"] = [len(list(nx.common_neighbors(G,u,v))) for (u,v) in enhanced.index]
    # M2: jaccard_coefficient
    #list(nx.jaccard_coefficient(G, ebunch=[(107,348)]))
    # M3: resouce allocation index
    #list(nx.resource_allocation_index(G, ebunch=[(107,348)]))
    # M4: Adamic Adar index
    #list(nx.adamic_adar_index(G, ebunch=[(107,348)]))
    # M5: Preferential Attachment
    pa = pd.DataFrame(nx.preferential_attachment(G))
    pa.columns = ['u','v','pa']
    pa['edge'] = list(zip(pa.u, pa.v))
    pa.set_index(pa['edge'],inplace=True)
    pa.drop(['u','v','edge'],axis=1,inplace=True)
    enhanced = enhanced.join(pa, how='left')
    # M6: Community Common Neighbors
    #list(nx.cn_soundarajan_hopcroft(G, ebunch=[(107,348)], community='Department'))
    # M7: Community Resource Allocation
    #list(nx.ra_index_soundarajan_hopcroft(G, ebunch=[(107,348)], community='Department'))
    
    X_train = enhanced.dropna()
    y_train = X_train['Future Connection']
    X_train.drop(['Future Connection'],axis=1,inplace=True)
    
    X_test = enhanced[enhanced['Future Connection'].isnull()]
    X_test.drop(['Future Connection'],axis=1,inplace=True)
    
    model = LogisticRegression()
    model = model.fit(X_train, y_train)
    
    X_test['proba'] = model.predict_proba(X_test)[:, 1]
    
    return X_test['proba']
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.038540
(542, 751)    0.014637
(20, 426)     0.641230
(50, 989)     0.014916
(942, 986)    0.015221
(324, 857)    0.014877
(13, 710)     0.164049
(19, 271)     0.108621
(319, 878)    0.015014
(659, 707)    0.014666
(49, 843)     0.015102
(208, 893)    0.014733
(377, 469)    0.006421
(405, 999)    0.026935
(129, 740)    0.022686
(292, 618)    0.023325
(239, 689)    0.015033
(359, 373)    0.009146
(53, 523)     0.040090
(276, 984)    0.015087
(202, 997)    0.015147
(604, 619)    0.049954
(270, 911)    0.015029
(261, 481)    0.080042
(200, 450)    0.856947
(213, 634)    0.014551
(644, 735)    0.049097
(346, 553)    0.013875
(521, 738)    0.012638
(422, 953)    0.022221
                ...   
(672, 848)    0.015029
(28, 127)     0.972458
(202, 661)    0.013762
(54, 195)     0.999926
(295, 864)    0.014834
(814, 936)    0.014594
(839, 874)    0.015221
(139, 843)    0.014858
(461, 544)    0.011822
(68, 487)     0.011684
(622, 932)    0.014766
(504, 936)    0.019094
(479, 528) 